In [ ]:
# default_exp ar

In [ ]:
# hide
import os
notebooks_dir = os.getcwd()
project_dir = os.path.dirname(notebooks_dir)

import sys
sys.path.append(project_dir)

# A Trader Robot
> Which Stabilize Fiat Value Curve of Crypto Currency

## Parameters

In [ ]:
# exports
TEST_RATIO = 1

In [ ]:
# exports
NOTIFY_RATE = 60 * 8
BACKUP_RATE = 1

## Find Symbols

In [ ]:
# exports
from decimal import Decimal

CRYPTO_SYMBOL = 'AR'
FIAT_SYMBOL = 'USDT'
MAX_USED_FIAT_MONEY_LIMIT = Decimal('1000')

In [ ]:
# exporti
from ccstabilizer import MXC
from ccstabilizer import secrets
from ccstabilizer import Fetcher
from ccstabilizer import Notifier
from ccstabilizer import Trader
from ccstabilizer import Status

In [ ]:
# exporti
exchange = MXC()
fetcher = Fetcher(exchange)
notifier = Notifier(channel_name='mxc', name='Launcher')

# notifier.send_slack(
#     f'{CRYPTO_SYMBOL}-{FIAT_SYMBOL} Detecting started\n', 'Power by https://jhub.name/', 'good'
# )

trading_spec = {}
while True:
    exchange.update_trading_specifications()
    trading_spec = fetcher.get_trading_spec(CRYPTO_SYMBOL, FIAT_SYMBOL)
    if trading_spec.get('liquid', False) == True:
        break
    else:
        time.sleep(Trader.SAMPLE_INTERVAL)

notifier.send_slack(
    f'{CRYPTO_SYMBOL}-{FIAT_SYMBOL} detected\n', 'Power by https://jhub.name/', 'good'
)

status = Status(
    robot_name = f'{CRYPTO_SYMBOL} Robot',
    crypto_symbol = CRYPTO_SYMBOL,
    fiat_symbol = FIAT_SYMBOL,
    trade_unit = trading_spec.get('min_trade_unit', 1),
    max_used_fiat_money_limit = MAX_USED_FIAT_MONEY_LIMIT,
)
status.read()

## Start Trading
### ToDo: Concurrent Control of Robots

In [ ]:
# exporti
from ccstabilizer import BookKeeper
from ccstabilizer import Trader

In [ ]:
# exporti
import time


with BookKeeper(exchange, [status]) as bookkeeper:

#     from unittest.mock import Mock
#     bookkeeper.prv.order = Mock()

    trader = Trader(
        exchange = exchange,
        status = status,
        gainable_unit_cc_sold_ratio = Decimal('0.236'),
        lossable_unit_cc_bought_ratio = Decimal('0.786'),
        min_trade_fiat_price = Decimal('0'),
        max_trade_fiat_price = Decimal('Infinity')
    )
    notifier = Notifier(channel_name='mxc', name=status.crypto_symbol)

    notifier.send_slack(
        f'{status.robot_name} launched\n' f'{status.get_robot_title()}', 'Power by https://jhub.name/', 'good'
    )

    while __name__ == '__main__':

        cooling_interval, trade_type, unit_amount = trader.check_and_trade()

        bookkeeper.fsh.write(f'{status}\n')

#         new_status = bookkeeper.estimate_functions.get(trade_type, lambda unit_amount: status)(unit_amount)
        new_status = bookkeeper.estimate_status_list()[0]

        if new_status is not status:

            status.update(new_status)

            bookkeeper.fth.write(f'{status}\n')
            status.write()

            notifier.send_slack(
                f'{bookkeeper.last_transaction} => {status.get_robot_title()}',
                'Power by https://jhub.name/', 'good' if status.total_gained_fiat_money >= 0 else 'danger'
            )

            notifier.send_slack(
                f'{status} => Support level is {trader.min_unit_cc_trade_fiat_money} {status.fiat_symbol}.',
                'Power by https://jhub.name/', 'good' if status.get_total_gain_fiat_money() >= 0 else 'danger'
            )

#             if status.bought_unit_amount == 0:
#                 break

            if status.total_gained_fiat_money < -Trader.MAX_LOST_JPY:
                break

        else:

            if status.sample_number % Trader.TRADE_RATE == 0:
                bookkeeper.fth.write(f'{status}\n')

            if status.sample_number % BACKUP_RATE == 0:
                status.write()

            if status.sample_number % NOTIFY_RATE == 0:
                notifier.send_slack(
                    f'{status} => Support level is {trader.min_unit_cc_trade_fiat_money} {status.fiat_symbol}.',
                    'Power by https://jhub.name/', 'good' if status.get_total_gain_fiat_money() > 0 else 'danger'
                )

        status.sample_number += 1
        time.sleep(cooling_interval / TEST_RATIO)

    notifier.send_slack(
        f'{status.robot_name} terminated\n' f'{status.get_robot_title()}', 'Power by https://jhub.name/', 'good'
    )

In [ ]:
# hide
from nbdev.export import notebook2script; notebook2script()